<a href="https://colab.research.google.com/github/leonidsanov/apache_spark/blob/main/homework3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашняя работа к третьему семинару

Условия:

есть набор данных о продажах продуктов с информацией о дате продаж, категории продукта, количестве и выручке от продаж.

Используя Apache Spark, загрузите предоставленный набор данных в DataFrame (пример данных ниже).

("2023-11-20", "Electronics", 100, 12000),

("2023-11-21", "Electronics", 110, 13000),

("2023-11-22", "Electronics", 105, 12500),

("2023-11-20", "Clothing", 300, 15000),

("2023-11-21", "Clothing", 280, 14000),

("2023-11-22", "Clothing", 320, 16000),

("2023-11-20", "Books", 150, 9000),

("2023-11-21", "Books", 200, 12000),

("2023-11-22", "Books", 180, 10000)

Столбцы: "date", "category", "quantity", "revenue".

С использованием оконных функций, рассчитайте среднее выручки от продаж для каждой категории продукта.
Примените операцию pivot для того, чтобы преобразовать полученные данные таким образом, чтобы в качестве строк были категории продуктов, в качестве столбцов были дни, а значениями были средние значения выручки от продаж за соответствующий день.

In [ ]:
!pip install pyspark >> None

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg
from pyspark.sql.window import Window

In [ ]:
# Инициализация SparkSession
spark = SparkSession.builder.appName("SalesDataAnalysis").getOrCreate()

In [ ]:
# Создание DataFrame с данными о продажах
sales_data = [
    ("2023-11-20", "Electronics", 100, 12000),
    ("2023-11-21", "Electronics", 110, 13000),
    ("2023-11-22", "Electronics", 105, 12500),
    ("2023-11-20", "Clothing", 300, 15000),
    ("2023-11-21", "Clothing", 280, 14000),
    ("2023-11-22", "Clothing", 320, 16000),
    ("2023-11-20", "Books", 150, 9000),
    ("2023-11-21", "Books", 200, 12000),
    ("2023-11-22", "Books", 180, 10000)
]

In [ ]:
# Определение схемы DataFrame
schema = ["date", "category", "quantity", "revenue"]

In [ ]:
# Загрузка данных в DataFrame
df = spark.createDataFrame(data=sales_data, schema=schema)

In [ ]:
# Определение оконной функции для расчёта средней выручки
windowSpec = Window.partitionBy("category")

In [ ]:
# Расчёт средней выручки для каждой категории
average_revenue = df.withColumn("avg_revenue", avg(col("revenue")).over(windowSpec))

In [ ]:
# Применение операции pivot
pivot_df = average_revenue.groupBy("category").pivot("date").avg("revenue")

In [ ]:
# Показать результат
pivot_df.show()

+-----------+----------+----------+----------+
|   category|2023-11-20|2023-11-21|2023-11-22|
+-----------+----------+----------+----------+
|Electronics|   12000.0|   13000.0|   12500.0|
|   Clothing|   15000.0|   14000.0|   16000.0|
|      Books|    9000.0|   12000.0|   10000.0|
+-----------+----------+----------+----------+



In [ ]:
# Завершение работы SparkSession
spark.stop()